In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn

print("✅ Libraries imported successfully today at:", datetime.now())
print(f"📅 Training started at: {datetime.now()}")

✅ Libraries imported successfully
📅 Training started at: 2026-01-09 10:25:37.314399


In [11]:
# Load training data from Lakehouse
print("📂 Loading data from Lakehouse...")

# Local execution: read from CSV directly
# Fabric execution: use spark.read (uncomment lines below when running in Fabric)
try:
    # Try Fabric/Spark approach first
    data = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/silver/customer_features/sample_customer_data.csv") \
        .toPandas()
    print("✅ Loaded from Fabric Lakehouse via Spark")
except:
    # Fallback to local CSV file (go up one directory from notebooks/)
    data = pd.read_csv("../sample_customer_data.csv")
    print("✅ Loaded from local CSV file")

print(f"✅ Loaded {len(data)} customer records")
print(f"\n📊 Dataset shape: {data.shape}")
print(f"\n🎯 Churn distribution:\n{data['churn'].value_counts()}")

📂 Loading data from Lakehouse...
✅ Loaded from local CSV file
✅ Loaded 100 customer records

📊 Dataset shape: (100, 7)

🎯 Churn distribution:
churn
0    69
1    31
Name: count, dtype: int64


In [12]:
# Feature engineering
print("🔧 Engineering features...")

# ⚠️ INTENTIONAL ERROR: Division by zero when customer_age_days is 0
data['purchase_frequency'] = data['total_purchases'] / (data['customer_age_days'] / 30)
data['engagement_score'] = (data['total_purchases'] * data['avg_purchase_value']) / data['customer_age_days']
data['recency_score'] = 1 / (data['days_since_last_purchase'] + 1)

feature_columns = [
    'total_purchases', 'avg_purchase_value', 'days_since_last_purchase',
    'customer_age_days', 'support_tickets', 'purchase_frequency',
    'engagement_score', 'recency_score'
]

X = data[feature_columns]
y = data['churn']

print(f"✅ Features prepared: {len(feature_columns)} features")

🔧 Engineering features...
✅ Features prepared: 8 features


In [13]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Training set: {len(X_train)} samples")
print(f"📊 Test set: {len(X_test)} samples")

📊 Training set: 80 samples
📊 Test set: 20 samples


In [14]:
# Model training with MLflow tracking
print("\n🤖 Training Random Forest model...")

mlflow.set_experiment("NVR_Customer_Churn_Prediction")

with mlflow.start_run(run_name=f"rf_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
    # Model parameters
    params = {
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 5,
        'random_state': 42
    }
    
    # Log parameters
    mlflow.log_params(params)
    
    # Train model
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    
    print("✅ Model training completed")
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "random_forest_model")
    
    print("\n📊 Model Performance Metrics:")
    print("=" * 40)
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print("=" * 40)


🤖 Training Random Forest model...
✅ Model training completed


2026/01/09 10:26:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



📊 Model Performance Metrics:
  Accuracy:  1.0000
  Precision: 1.0000
  Recall:    1.0000
  F1 Score:  1.0000


In [15]:
# Feature importance analysis
print("\n🔍 Feature Importance:")
print("=" * 40)

feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in feature_importance.iterrows():
    print(f"  {row['feature']:30s} {row['importance']:.4f}")

print("=" * 40)


🔍 Feature Importance:
  days_since_last_purchase       0.2225
  customer_age_days              0.1987
  avg_purchase_value             0.1975
  recency_score                  0.1158
  total_purchases                0.1025
  support_tickets                0.0920
  engagement_score               0.0703
  purchase_frequency             0.0007


In [16]:
# Model validation summary
print("\n✅ Model Training Pipeline Completed")
print("=" * 50)
print(f"  Model Type: Random Forest Classifier")
print(f"  Training Samples: {len(X_train)}")
print(f"  Test Samples: {len(X_test)}")
print(f"  Features: {len(feature_columns)}")
print(f"  Model Accuracy: {accuracy:.2%}")
print(f"  Completed at: {datetime.now()}")
print("=" * 50)
print("\n🚀 Model ready for deployment!")


✅ Model Training Pipeline Completed
  Model Type: Random Forest Classifier
  Training Samples: 80
  Test Samples: 20
  Features: 8
  Model Accuracy: 100.00%
  Completed at: 2026-01-09 10:27:17.623769

🚀 Model ready for deployment!


In [10]:
import os
from datetime import datetime
# Enhanced logging for production monitoring
print("=" * 50)
print("🚀 Model Training v2.0 - Enhanced Logging")
print("=" * 50)
print(f"Training started: {datetime.now()}")
print(f"Environment: {os.environ.get('ENVIRONMENT', 'dev')}")
print(f"Model version: 2.0")
print("=" * 50)

🚀 Model Training v2.0 - Enhanced Logging
Training started: 2026-01-09 10:25:57.086557
Environment: dev
Model version: 2.0
